# 2. Относительные изменения цен

### Подключение необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import csv

#### Функция для считывания и обработки csv-файлов с данными тикеров и вычисления относительных изменений

In [2]:
def analysis(ticker):
    # Считываем csv-файл с данными тикера    
    ticker_data = pd.read_csv(ticker + '.csv', sep = ',')

    # Форматируем файл, удаляем лишние столбцы
    cols = ['Open','High','Low','Adj Close','Volume']
    ticker_info = ticker_data.drop(cols,axis=1)
    
    # Форматируем файл, "срезаем" дату до значения года
    year_list = [(str(ticker_info['Date'][i])[:4]) for i in range(len(ticker_info['Date'])) ]
    ticker_info['Date'] = year_list
    
    # Создаем и форматируем список изменений за весь рассматриваемый период
    change_value_list = []
    for i in range(len(ticker_info['Close'])-1):
        change_value_list.append(round(ticker_info['Close'][i+1]/ticker_info['Close'][i]*100-100, 3 ))
    #for i in range (len(change_value_list)):
    #    change_value_list[i] = float(change_value_list[i].replace(",","."))

    # Создаем список количества торговых дней за каждый год
    trading_days = [0 * i for i in range(2012,2022)]
    for year in range(2012,2022):
        for i in range(len(ticker_info['Date'])):
            if int(ticker_data['Date'][i][:4]) == year:
                trading_days[year-2012] += 1
              
    # Создаем пустые списки для дальнейшего заполнения максимальными изменениями
    change_up_list = []
    change_down_list = []

    # Создаём вспомогательный список количества торговых дней, начиная с нуля
    'AAPL: [0, 250, 252, 252, 252, 253, 251, 251, 252, 253, 252]'
    gaps_list = [0]
    for j in range(len(trading_days)):
        gaps_list.append(trading_days[j])
    
    # Создаём вспомогательный список начальных значений за каждый год из общего списка
    'AAPL: [0, 250, 502, 754, 1006, 1259, 1510, 1761, 2013, 2266, 2518]'
    num_list = []
    start_of_year = 0
    for i in range(len(gaps_list)):
        start_of_year += gaps_list[i]
        num_list.append(start_of_year)

    # Создаем пустой список для дальнейшего заполнения изменениями за каждый год
    annual_changes_list = []
    
    # Заполняем список списками изменений за каждый год
    for i in range(10):
        annual_changes_list.append(change_value_list)
    for i in range(len(trading_days)):
        annual_changes_list[i]=annual_changes_list[i][num_list[i]:num_list[i+1]]

    # Заполняем списки максимальными изменениями вверх и вниз за каждый год
    for i in range (len(annual_changes_list)):
        max = 0
        min = 0
        for j in range(len(annual_changes_list[i])):
            if float(annual_changes_list[i][j]) > max:
                max = annual_changes_list[i][j]
            if float(annual_changes_list[i][j]) < min:
                min = annual_changes_list[i][j]
        change_up_list.append(max)
        change_down_list.append(min)

    # Создаём DataFrame из списков максимумов и минимумов
    change_data = pd.DataFrame({'Изменение вверх': change_up_list, 'Изменение вниз': change_down_list})
    change_data = change_data.transpose()
    
    return change_data

### Формирование DataFrame'ов из максимальных относительных изменений

In [3]:
# Создаём пустые DataFrame
rising_up_data = pd.DataFrame()
rising_down_data = pd.DataFrame()

# Создаём список названий всех используемых тикеров
tickers = ['AAPL','ADBE','AMD','CSCO','FB','GOOGL','INTC','MSFT','NFLX','NVDA']

# Заполняем DataFrame значениями по каждому тикеру
for ticker in tickers:
    t = analysis(ticker)
    
    up = t.transpose()['Изменение вверх']
    up = pd.DataFrame(up).transpose()
    rising_up_data = rising_up_data.append(up, ignore_index=True)
    
    down = t.transpose()['Изменение вниз']
    down = pd.DataFrame(down).transpose()
    rising_down_data = rising_down_data.append(down, ignore_index=True)

# Изменяем названия строк и столбцов
rising_up_data.columns = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
rising_down_data.columns = ['2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']
rising_up_data.index = rising_down_data.index = tickers

### Максимальные дневные повышения цен в %

In [4]:
# rising_up_data - Вывод таблицы максимальных дневных повышений в %

### Максимальные дневные понижения цен в %

In [5]:
# rising_down_data - Вывод таблицы максимальных дневных понижений в %

### Сохраним данные в csv-файле 

In [21]:
years = ['2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']
tickers = ['AAPL','ADBE','AMD','CSCO','FB','GOOGL','INTC','MSFT','NFLX','NVDA']
line = [['-']]

In [22]:
name_table_up = "Таблица 3. Максимальные дневные повышения цен в %.csv"
    
for i in range(len(years)): 
    line[0].append(years[i])
with open(name_table_up, "w", newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter=';')
    writer.writerows(line)
    for ticker in tickers:
        line = [[]]
        line[0].append(ticker)
        for year in years:
            line[0].append(str(rising_up_data[year][ticker]).replace('.', ','))
        writer.writerows(line)

In [23]:
name_table_down = "Таблица 4. Максимальные дневные понижения цен в %.csv"

for i in range(len(years)): 
    line[0].append(years[i])
with open(name_table_down, "w", newline='') as csv_file:
    writer = csv.writer(csv_file,delimiter=';')
    writer.writerows(line)
    for ticker in tickers:
        line = [[]]
        line[0].append(ticker)
        for year in years:
            line[0].append(str(rising_down_data[year][ticker]).replace('.', ','))
        writer.writerows(line)